# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [7]:
DATA_PATH = os.path.join('MP_Data') 

actions = np.array(['tea','teach','technology'])

no_sequences = 30

sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [45]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    

    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [46]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'tea': 0, 'teach': 1, 'technology': 2}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
y = to_categorical(labels).astype(int)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
y_test.shape

(5, 3)

# 7. Build and Train LSTM Neural Network

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [60]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [61]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [62]:
res = [.7, 0.2, 0.1]

In [63]:
actions[np.argmax(res)]

'tea'

In [64]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [65]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 224ms/step - loss: 1.1309 - categorical_accuracy: 0.3176
Epoch 2/2000
3/3 [==============================] - 0s 59ms/step - loss: 2.6477 - categorical_accuracy: 0.3412
Epoch 3/2000
3/3 [==============================] - 0s 60ms/step - loss: 1.1869 - categorical_accuracy: 0.3294
Epoch 4/2000
3/3 [==============================] - 0s 64ms/step - loss: 1.5482 - categorical_accuracy: 0.3294
Epoch 5/2000
3/3 [==============================] - 0s 57ms/step - loss: 1.2254 - categorical_accuracy: 0.3647
Epoch 6/2000
3/3 [==============================] - 0s 58ms/step - loss: 1.0809 - categorical_accuracy: 0.3529
Epoch 7/2000
3/3 [==============================] - 0s 59ms/step - loss: 1.1006 - categorical_accuracy: 0.2824
Epoch 8/2000
3/3 [==============================] - 0s 58ms/step - loss: 1.1045 - categorical_accuracy: 0.3529
Epoch 9/2000
3/3 [==============================] - 0s 57ms/step - loss: 0.9576 - categorical_accuracy: 0.6118


3/3 [==============================] - 0s 56ms/step - loss: 0.4866 - categorical_accuracy: 0.7412
Epoch 75/2000
3/3 [==============================] - 0s 56ms/step - loss: 0.4689 - categorical_accuracy: 0.7412
Epoch 76/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.4536 - categorical_accuracy: 0.8118
Epoch 77/2000
3/3 [==============================] - 0s 101ms/step - loss: 0.4530 - categorical_accuracy: 0.7882
Epoch 78/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.4281 - categorical_accuracy: 0.8000
Epoch 79/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.4177 - categorical_accuracy: 0.7882
Epoch 80/2000
3/3 [==============================] - 0s 105ms/step - loss: 0.4185 - categorical_accuracy: 0.8471
Epoch 81/2000
3/3 [==============================] - 0s 102ms/step - loss: 0.4010 - categorical_accuracy: 0.8235
Epoch 82/2000
3/3 [==============================] - 0s 101ms/step - loss: 0.3826 - categorical_accuracy: 0.8706


3/3 [==============================] - 0s 70ms/step - loss: 0.3307 - categorical_accuracy: 0.7882
Epoch 219/2000
3/3 [==============================] - 0s 69ms/step - loss: 0.2749 - categorical_accuracy: 0.9529
Epoch 220/2000
3/3 [==============================] - 0s 70ms/step - loss: 0.2370 - categorical_accuracy: 0.9765
Epoch 221/2000
3/3 [==============================] - 0s 69ms/step - loss: 0.2267 - categorical_accuracy: 0.9529
Epoch 222/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.2153 - categorical_accuracy: 0.9529
Epoch 223/2000
3/3 [==============================] - 0s 69ms/step - loss: 0.1946 - categorical_accuracy: 0.9529
Epoch 224/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.1782 - categorical_accuracy: 0.9529
Epoch 225/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.1643 - categorical_accuracy: 0.9529
Epoch 226/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.1510 - categorical_accuracy: 0.952

3/3 [==============================] - 0s 67ms/step - loss: 0.0050 - categorical_accuracy: 1.0000
Epoch 363/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.0042 - categorical_accuracy: 1.0000
Epoch 364/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.0045 - categorical_accuracy: 1.0000
Epoch 365/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.0037 - categorical_accuracy: 1.0000
Epoch 366/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.0025 - categorical_accuracy: 1.0000
Epoch 367/2000
3/3 [==============================] - 0s 69ms/step - loss: 0.0024 - categorical_accuracy: 1.0000
Epoch 368/2000
3/3 [==============================] - 0s 68ms/step - loss: 0.0022 - categorical_accuracy: 1.0000
Epoch 369/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 370/2000
3/3 [==============================] - 0s 67ms/step - loss: 0.0018 - categorical_accuracy: 1.000

3/3 [==============================] - 0s 67ms/step - loss: 1.0600e-04 - categorical_accuracy: 1.0000
Epoch 503/2000
3/3 [==============================] - 0s 68ms/step - loss: 1.0487e-04 - categorical_accuracy: 1.0000
Epoch 504/2000
3/3 [==============================] - 0s 69ms/step - loss: 1.0397e-04 - categorical_accuracy: 1.0000
Epoch 505/2000
3/3 [==============================] - 0s 67ms/step - loss: 1.0295e-04 - categorical_accuracy: 1.0000
Epoch 506/2000
3/3 [==============================] - 0s 71ms/step - loss: 1.0181e-04 - categorical_accuracy: 1.0000
Epoch 507/2000
3/3 [==============================] - 0s 68ms/step - loss: 1.0059e-04 - categorical_accuracy: 1.0000
Epoch 508/2000
3/3 [==============================] - 0s 68ms/step - loss: 9.9600e-05 - categorical_accuracy: 1.0000
Epoch 509/2000
3/3 [==============================] - 0s 76ms/step - loss: 9.8671e-05 - categorical_accuracy: 1.0000
Epoch 510/2000
3/3 [==============================] - 0s 82ms/step - loss: 9.78

...

3/3 [==============================] - 0s 68ms/step - loss: 9.0905e-06 - categorical_accuracy: 1.0000
Epoch 923/2000
3/3 [==============================] - 0s 67ms/step - loss: 9.0541e-06 - categorical_accuracy: 1.0000
Epoch 924/2000
3/3 [==============================] - 0s 68ms/step - loss: 9.0148e-06 - categorical_accuracy: 1.0000
Epoch 925/2000
3/3 [==============================] - 0s 69ms/step - loss: 8.9797e-06 - categorical_accuracy: 1.0000
Epoch 926/2000
3/3 [==============================] - 0s 67ms/step - loss: 8.9419e-06 - categorical_accuracy: 1.0000
Epoch 927/2000
3/3 [==============================] - 0s 68ms/step - loss: 8.9138e-06 - categorical_accuracy: 1.0000
Epoch 928/2000
3/3 [==============================] - 0s 69ms/step - loss: 8.8662e-06 - categorical_accuracy: 1.0000
Epoch 929/2000
3/3 [==============================] - 0s 67ms/step - loss: 8.8311e-06 - categorical_accuracy: 1.0000
Epoch 930/2000
3/3 [==============================] - 0s 68ms/step - loss: 8.79

3/3 [==============================] - 0s 66ms/step - loss: 5.3713e-06 - categorical_accuracy: 1.0000
Epoch 1062/2000
3/3 [==============================] - 0s 64ms/step - loss: 5.3517e-06 - categorical_accuracy: 1.0000
Epoch 1063/2000
3/3 [==============================] - 0s 72ms/step - loss: 5.3377e-06 - categorical_accuracy: 1.0000
Epoch 1064/2000
3/3 [==============================] - 0s 71ms/step - loss: 5.3082e-06 - categorical_accuracy: 1.0000
Epoch 1065/2000
3/3 [==============================] - 0s 71ms/step - loss: 5.2928e-06 - categorical_accuracy: 1.0000
Epoch 1066/2000
3/3 [==============================] - 0s 72ms/step - loss: 5.2732e-06 - categorical_accuracy: 1.0000
Epoch 1067/2000
3/3 [==============================] - 0s 67ms/step - loss: 5.2521e-06 - categorical_accuracy: 1.0000
Epoch 1068/2000
3/3 [==============================] - 0s 73ms/step - loss: 5.2311e-06 - categorical_accuracy: 1.0000
Epoch 1069/2000
3/3 [==============================] - 0s 85ms/step - lo

3/3 [==============================] - 0s 81ms/step - loss: 3.3196e-06 - categorical_accuracy: 1.0000
Epoch 1200/2000
3/3 [==============================] - 0s 70ms/step - loss: 3.3098e-06 - categorical_accuracy: 1.0000
Epoch 1201/2000
3/3 [==============================] - 0s 71ms/step - loss: 3.2901e-06 - categorical_accuracy: 1.0000
Epoch 1202/2000
3/3 [==============================] - 0s 67ms/step - loss: 3.2789e-06 - categorical_accuracy: 1.0000
Epoch 1203/2000
3/3 [==============================] - 0s 71ms/step - loss: 3.2705e-06 - categorical_accuracy: 1.0000
Epoch 1204/2000
3/3 [==============================] - 0s 68ms/step - loss: 3.2593e-06 - categorical_accuracy: 1.0000
Epoch 1205/2000
3/3 [==============================] - 0s 70ms/step - loss: 3.2495e-06 - categorical_accuracy: 1.0000
Epoch 1206/2000
3/3 [==============================] - 0s 68ms/step - loss: 3.2397e-06 - categorical_accuracy: 1.0000
Epoch 1207/2000
3/3 [==============================] - 0s 69ms/step - lo

3/3 [==============================] - 0s 70ms/step - loss: 2.0476e-06 - categorical_accuracy: 1.0000
Epoch 1338/2000
3/3 [==============================] - 0s 70ms/step - loss: 2.0392e-06 - categorical_accuracy: 1.0000
Epoch 1339/2000
3/3 [==============================] - 0s 69ms/step - loss: 2.0322e-06 - categorical_accuracy: 1.0000
Epoch 1340/2000
3/3 [==============================] - 0s 68ms/step - loss: 2.0279e-06 - categorical_accuracy: 1.0000
Epoch 1341/2000
3/3 [==============================] - 0s 69ms/step - loss: 2.0223e-06 - categorical_accuracy: 1.0000
Epoch 1342/2000
3/3 [==============================] - 0s 69ms/step - loss: 2.0139e-06 - categorical_accuracy: 1.0000
Epoch 1343/2000
3/3 [==============================] - 0s 70ms/step - loss: 2.0097e-06 - categorical_accuracy: 1.0000
Epoch 1344/2000
3/3 [==============================] - 0s 67ms/step - loss: 1.9957e-06 - categorical_accuracy: 1.0000
Epoch 1345/2000
3/3 [==============================] - 0s 70ms/step - lo

KeyboardInterrupt: 

In [66]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

# 8. Make Predictions

In [67]:
res = model.predict(X_test)

In [68]:
actions[np.argmax(res[4])]

'teach'

In [69]:
actions[np.argmax(y_test[4])]

'teach'

# 9. Save Weights

In [101]:
model.save('action.h5')

In [102]:
del model

In [21]:
from tensorflow import keras
model = keras.models.load_model('action.h5')

In [22]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [23]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [24]:
yhat = model.predict(X_test)

In [25]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [26]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[2, 0],
        [0, 3]]], dtype=int64)

In [27]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [28]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()   
    return output_frame

In [39]:
import math
sequence = []
sentence = []
predictions = []
threshold = 0.3
frameRate = cap.get(5)

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if(ret==True):
            image, results = mediapipe_detection(frame, holistic)
            if not (results.left_hand_landmarks or results.right_hand_landmarks):
                continue
            draw_styled_landmarks(image, results)
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                if len(sentence) > 3: 
                    sentence = sentence[-3:]
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            i=0
            while(len(sequence)<30):
                sequence.append(sequence[i])
                i+=1
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predictions.append(np.argmax(res))
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 
                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 30: 
                    sentence = sentence[-3:]
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

tea
tea
tea
tea
tea
technology
technology
technology
technology
technology
technology
technology
teach
teach
teach
teach
teach
tea
tea
tea
tea
tea
tea
tea
tea
tea
tea
tea
tea
tea
tea
tea
tea
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
teach
tea
tea
tea
tea
tea
tea
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology
technology


In [34]:
cap.release()
cv2.destroyAllWindows()